In [16]:
import json
import string
import requests
from requests.exceptions import Timeout
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from scipy.stats import linregress
import csv 
import io


In [47]:
######get fatality data 
#beginning year

#/FARSData/GetFARSData?dataset=Accident&FromYear=2014&ToYear=2016&State=1&format=csv

dataset = ['Accident','CEVENT','DRUGS','DISTRACT','DRIMPAIR','MANEUVER','PERSON','VIOLATION','VISION']
baseurl = 'https://crashviewer.nhtsa.dot.gov/CrashAPI/FARSData/GetFARSData'

for x in dataset:
    yearloop = 2010
    params = {'states':'1,51',
            'FromYear':yearloop,
            'ToYear':yearloop,
            'dataset':x,
            'format':'csv'}
    for i in range (12):
        #wait up to 70 seconds for responding data
        try:
            response = requests.get(baseurl,params=params,timeout=(10,70))
        except Timeout:
            print('The request timed out')

        if response.status_code !=200:
            print('There is error')
        else:
            #get response as CSV
            response.encoding = response.apparent_encoding
            #get data in string
            content = response.text
            print(yearloop)
            #convert string to pandas dataframe
            temp_data = pd.read_csv(io.StringIO(content))
            if i ==0:
                #response_json = response.iter_lines()
                #save data to fatality_data if 1st data
                fatality_data = pd.read_csv(io.StringIO(content))           
            else:
                #response_json = response.iter_lines()
                #append 2nd data point on ward to fatality_data
                temp_data = pd.read_csv(io.StringIO(content))
                fatality_data = pd.concat([fatality_data,temp_data],ignore_index=True) 
        #update parameter for next year
        yearloop += 1
        params = {'states':'1,51',
            'FromYear':yearloop,
            'ToYear':yearloop,
            'dataset':x,
            'format':'csv'}

    #export final data to csv
    fatality_data.to_csv(f'data/{x}.csv',index=False)

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2010


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2011
2012
2013
2014
2015


c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (17,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2016
2017
2018
2019
2020
2021
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
